In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import os
from multiprocessing import Pool
import pymysql
import pandas as pd


## Buy-it-now listings of <"canon eos 5d">.

In [4]:
def getLinks(qrkw, page, ipg=100, BIN=True):
    """
    The function takes search item and page number, and returns a list of links for sponsored and nonsponsored items.
    
    PARAM qrkw: str, search item;
    PARAM page: int, page number;
    PARAM ipg: int, item per page, default=100;
    PARAM BIN: boolean, buy-it-now listing, default=True;
    
    RETURN a list of url links for sponsored and nonsponsored items on the selected page.
    """
    if BIN:
        LH_BIN = 1
    else:
        LH_BIN = 0
    
    base_url = "https://www.ebay.com/sch/i.html?"
    url = base_url + "_nkw=" + qrkw + "&LH_BIN=" + str(LH_BIN) + "&_ipg=" + str(ipg) + "&_pgn=" + str(page)
    r = requests.get(url, headers={'user-agent': 'Mozilla/5.0'})
    soup_r = BeautifulSoup(r.content)
    links = soup_r.find_all("a", class_="s-item__link")
    sponsored_links=[]
    nonsponsored_links=[]
    for link in links:
        simp_link = re.sub("(.*)\\?.*", r"\1", link['href'])
        tag_text = link.find_all('span')[0].get_text()
        if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*', tag_text):
            sponsored_links.append(simp_link)
        else:
            nonsponsored_links.append(simp_link)
            
    return sponsored_links, nonsponsored_links

In [5]:
# Get all the sponsored and nonsponsored URLs of a page, save them into a list.
kw = "canon eos 5d"
sponsored_each = []
nonsponsored_each = []
for i in range(1,11):
    temp_sp, temp_nsp = getLinks(kw, i)
    sponsored_each.append(temp_sp)
    nonsponsored_each.append(temp_nsp)
    print("URLs retrieved, Page",i)
    time.sleep(2)

URLs retrieved, Page 1
URLs retrieved, Page 2
URLs retrieved, Page 3
URLs retrieved, Page 4
URLs retrieved, Page 5
URLs retrieved, Page 6
URLs retrieved, Page 7
URLs retrieved, Page 8
URLs retrieved, Page 9
URLs retrieved, Page 10


In [48]:
# Write all retrieved URLs into two txt files.
sponsored = []
for items in sponsored_each:
    for i in items:
        sponsored.append(i)
        
with open("sponsored.txt", "w") as file:
    for item in sponsored:
        file.write('%s\n' % item)
        
# Write all sponored and non-sponsored URLs into a txt file.
nonsponsored = []
for items in nonsponsored_each:
    for i in items:
        nonsponsored.append(i)
        
with open("non-sponsored.txt", "w") as file:
    for item in nonsponsored:
        file.write('%s\n' % item)

In [49]:
# Create directories for "sponsored" and "nonsponsored" pages.
try:
    path_sponsored = os.getcwd() + "/sponsored"
    path_nonsponsored = os.getcwd() + "/non-sponsored"
    os.makedirs(path_sponsored)
    os.makedirs(path_nonsponsored)
except OSError:
    print ("Creation of the directory failed")
else:
    print ("Successfully created the directory.")

Successfully created the directory.


In [50]:
def savehtm_sp(url):
    """
    The function takes the url and save it locally in a html file.
    
    PARAM url: str, url link;
    """
    try:
        item_id = re.search(r'[0-9]{12}', url)[0]
        soup_temp = BeautifulSoup(requests.get(url, headers={'user-agent': 'Mozilla/5.0'}).content) 
        with open(os.getcwd() + "/sponsored/" + item_id+".htm", "w") as f:
            f.write(str(soup_temp))
        #print("============================")
        #print(item_id,"Saved")
        time.sleep(2)
        
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))
    except KeyboardInterrupt:
        print("Someone closed the program")

In [51]:
# put all sponsored links into a list.
sponsored_links = []
with open("sponsored.txt", "r") as file:
    for link in file:
        sponsored_links.append(link.replace('\n',''))

In [52]:
# using parallel processing to save html in sponsored folder.
p = Pool(10)  # Processing 10 links at a time.
save_sptml = p.map(savehtm_sp, sponsored_links)
p.terminate()
p.join()

In [35]:
def savehtm_non(url):
    """
    The function takes the url and save it locally in a html file.
    
    PARAM url: str, url link;
    """
    try:
        item_id = re.search(r'[0-9]{12}', url)[0]
        soup_temp = BeautifulSoup(requests.get(url, headers={'user-agent': 'Mozilla/5.0'}).content) 
        with open(os.getcwd() + "/non-sponsored/" + item_id + ".htm", "w") as f:
            f.write(str(soup_temp))
        #print("============================")
        #print(item_id,"Saved")
        time.sleep(2)
    
    except requests.ConnectionError as e:
        print("OOPS!! Connection Error. Make sure you are connected to Internet. Technical Details given below.\n")
        print(str(e))
    except requests.Timeout as e:
        print("OOPS!! Timeout Error")
        print(str(e))
    except requests.RequestException as e:
        print("OOPS!! General Error")
        print(str(e))
    except KeyboardInterrupt:
        print("Someone closed the program")

In [53]:
# put all nonsponsored links into a list.
nonsponsored = []
with open("non-sponsored.txt", "r") as file:
    for link in file:
        nonsponsored.append(link.replace('\n',''))

In [64]:
# using parallel processing to save html in non-sponsored folder.
p = Pool(10)  # Processing 10 links at a time.
save_nsphtm = p.map(savehtm_non, nonsponsored)
p.close()
p.terminate()
p.join()

## Extract item information

In [230]:
def extractInfo(html):
    """
    The function takes the html file from sponsored and return a dictionary containing item information.
    
    PARAM str: html file name;
    
    RETURN dict: item_id, item_title, item_price, condition, items_sold, seller_name, seller_score,
                 best_offer, returns_allowed, shipping_cal, shipping_cost
    """
    # soup object to start with
    soup_pgn = BeautifulSoup(open(os.getcwd() + "/sponsored/" + html))
    
    # this for sponsored items only.
    sponsored = True
    
    # get item id from the html file name
    item_id = int(re.search(r'[0-9]{12}', html)[0])
    
    # get the item title by looking at the first header with id=itemTitle, stripping out unwanted characters
    item_title = soup_pgn.find('h1',{'id':'itemTitle'}).get_text().replace('Details about  \xa0','').replace('\xa0','')
    
    # look for price summary span with id=prcIsum, when it is not in USD, use price in span with id=convbinPrice
    # when it is in USD, just use the price; or when it is on sale, use discounted price
    # for both situations, convert US dollars into US cents
    if soup_pgn.find('span',{'id':'prcIsum'}):
        if soup_pgn.find('span',{'id':'convbinPrice'}):
            raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'convbinPrice'}).get_text().strip().replace(',',''))[0]
            item_price = int(float(raw_price_text)*100)
        else:
            raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'prcIsum'}).get_text().strip().replace(',',''))[0]
            item_price = int(float(raw_price_text)*100)
    else:
        raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'mm-saleDscPrc'}).get_text().strip().replace(',',''))[0]
        item_price = int(float(raw_price_text)*100)
    
    # look for condition in div with id=vi-itm-cond, if not listed, return None.
    condition = None if soup_pgn.find('div',{'id':'vi-itm-cond'})==None else soup_pgn.find('div',{'id':'vi-itm-cond'}).get_text().strip()
    
    # look for item sold in span with a dedicated class attribute, convert into int, if not listed, return None.
    items_sold = None if soup_pgn.find('span', class_='vi-qtyS vi-bboxrev-dsplblk vi-qty-vert-algn vi-qty-pur-lnk')==None else int(soup_pgn.find('span', class_='vi-qtyS vi-bboxrev-dsplblk vi-qty-vert-algn vi-qty-pur-lnk').get_text().strip().replace(' sold',''))
    
    # look for seller name in span with class=mbg-nww, if not listed, return None.
    seller_name = None if soup_pgn.find('span', class_='mbg-nw')==None else soup_pgn.find('span', class_='mbg-nw').get_text()
    
    # look for seller_score in span with class=mbg-l, convert into int, if not listed, return None.
    seller_score = None if soup_pgn.findChild('span', class_='mbg-l').find('a')==None else int(soup_pgn.findChild('span', class_='mbg-l').find('a').get_text().strip())
    
    # look for better_offer availability in a with id=boBtn_btn, if not listed, return False.
    best_offer = False if soup_pgn.find('a',{'id':'boBtn_btn'})==None else True
    
    # look at the banner in span with id=vi-ret-accrd-txt, if it says not accept, return False, else True.
    if soup_pgn.find('span',{'id':'vi-ret-accrd-txt'}):
        if soup_pgn.find('span',{'id':'vi-ret-accrd-txt'}).get_text().strip()=='Seller does not accept returns':
            returns_allowed = False
        else:
            returns_allowed = True
    else:
        returns_allowed = True 
    
    # look for the shipping summary in the span with id=shSummary, if it says Calculate, return True, else False
    shipping_cal = True if soup_pgn.find('span',{'id':'shSummary'}).find('a').get_text()=='Calculate' else False
    # if shipping needs to be calculated, shipping cost is None
    if shipping_cal:
        shipping_cost = None
    # if not, look at the span with id=fshippingCost, 
    # if it says FREE, shipping cost=0, if not, return the US cents value for shipping
    elif soup_pgn.find('span',{'id':'fshippingCost'}):
            if soup_pgn.find('span',{'id':'fshippingCost'}).get_text().strip()=='FREE':
                shipping_cost = 0
            else:
                raw_text = soup_pgn.find('span',{'id':'fshippingCost'}).get_text().strip().replace(',','')
                shipping_cost = int(float(re.search(r'[0-9]+.[0-9]+', raw_text)[0])*100)
    # if it says fast and free, shipping cost=0
    elif soup_pgn.find('span',class_='vi-fnf-ship-txt fnfgreen'):
        shipping_cost = 0
    # if it is not in USD, use id=convetedPriceId, and return shipping cost in US cents
    elif soup_pgn.find('span',{'id':'convetedPriceId'}):
        raw_text = soup_pgn.find('span',{'id':'convetedPriceId'}).get_text().strip().replace(',','')
        shipping_cost = int(float(re.search(r'[0-9]+.[0-9]+', raw_text)[0])*100)
    # other situations like Local pick ups and needs contact seller, return None.
    else:
        shipping_cost = None
    
    
    info_keys = ['sponsored', 'item_id', 'item_title', 'item_price', 'condition', 'items_sold', 'seller_name', 'seller_score',
                 'best_offer', 'returns_allowed', 'shipping_cal', 'shipping_cost']
    info_vals = [sponsored, item_id, item_title, item_price, condition, items_sold, seller_name, seller_score,
                 best_offer, returns_allowed, shipping_cal, shipping_cost]
    item_dict = {info_keys[i]: info_vals[i] for i in range(len(info_keys))}
    
    return item_dict

In [285]:
# sponsored items' info using parallel computing
sponsored_files = os.listdir(os.getcwd() + "/sponsored/")
p = Pool(10)  # Processing 10 files at a time.
sponsored_items_info = p.map(extractInfo, sponsored_files)
p.terminate()
p.join()
sponsored_items_info

[{'sponsored': True,
  'item_id': 202800626639,
  'item_title': 'Canon EOS 5D Mark IV DSLR Camera + EF 24-70mm IS USM Lens Dual Battery Power Kit',
  'item_price': 324900,
  'condition': 'New',
  'items_sold': None,
  'seller_name': 'Beach Camera',
  'seller_score': 354988,
  'best_offer': False,
  'returns_allowed': True,
  'shipping_cal': False,
  'shipping_cost': 0},
 {'sponsored': True,
  'item_id': 362370637186,
  'item_title': 'Canon EOS 5D Mark IV Full Frame Digital SLR Camera Body Shutter: 698',
  'item_price': 279999,
  'condition': 'Used',
  'items_sold': None,
  'seller_name': 'rongyuanxie',
  'seller_score': 165739,
  'best_offer': False,
  'returns_allowed': True,
  'shipping_cal': False,
  'shipping_cost': 0},
 {'sponsored': True,
  'item_id': 372880508624,
  'item_title': 'Canon EOS 5D Mark IV DSLR with 24-105mm USM Lens Bundle With Atomos Monitor Kit',
  'item_price': 344900,
  'condition': 'New',
  'items_sold': None,
  'seller_name': 'Adorama',
  'seller_score': 52555

In [2]:
def extractInfo_non(html):
    """
    The function takes the html file from non-sponsored and return a dictionary containing item information.
    
    PARAM str: html file name;
    
    RETURN dict: item_id, item_title, item_price, condition, items_sold, seller_name, seller_score,
                 best_offer, returns_allowed, shipping_cal, shipping_cost
    """
    soup_pgn = BeautifulSoup(open(os.getcwd() + "/non-sponsored/" + html))
    
    sponsored = False
    
    # get item id from the html file name
    item_id = int(re.search(r'[0-9]{12}', html)[0])
    
    # get the item title by looking at the first header with id=itemTitle, stripping out unwanted characters
    item_title = soup_pgn.find('h1',{'id':'itemTitle'}).get_text().replace('Details about  \xa0','').replace('\xa0','')
    
    # look for price summary span with id=prcIsum, when it is not in USD, use price in span with id=convbinPrice
    # when it is in USD, just use the price; or when it is on sale, use discounted price
    # for both situations, convert US dollars into US cents
    if soup_pgn.find('span',{'id':'prcIsum'}):
        if soup_pgn.find('span',{'id':'convbinPrice'}):
            raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'convbinPrice'}).get_text().strip().replace(',',''))[0]
            item_price = int(float(raw_price_text)*100)
        else:
            raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'prcIsum'}).get_text().strip().replace(',',''))[0]
            item_price = int(float(raw_price_text)*100)
    else:
        raw_price_text = re.search(r'[0-9]+.[0-9]+',soup_pgn.find('span',{'id':'mm-saleDscPrc'}).get_text().strip().replace(',',''))[0]
        item_price = int(float(raw_price_text)*100)
    
    # look for condition in div with id=vi-itm-cond, if not listed, return None.
    condition = None if soup_pgn.find('div',{'id':'vi-itm-cond'})==None else soup_pgn.find('div',{'id':'vi-itm-cond'}).get_text().strip()
    
    # look for item sold in span with a dedicated class attribute, convert into int, if not listed, return None.
    items_sold = None if soup_pgn.find('span', class_='vi-qtyS vi-bboxrev-dsplblk vi-qty-vert-algn vi-qty-pur-lnk')==None else int(soup_pgn.find('span', class_='vi-qtyS vi-bboxrev-dsplblk vi-qty-vert-algn vi-qty-pur-lnk').get_text().strip().replace(' sold',''))
    
    # look for seller name in span with class=mbg-nww, if not listed, return None.
    seller_name = None if soup_pgn.find('span', class_='mbg-nw')==None else soup_pgn.find('span', class_='mbg-nw').get_text()
    
    # look for seller_score in span with class=mbg-l, convert into int, if not listed, return None.
    seller_score = None if soup_pgn.findChild('span', class_='mbg-l').find('a')==None else int(soup_pgn.findChild('span', class_='mbg-l').find('a').get_text().strip())
    
    # look for better_offer availability in a with id=boBtn_btn, if not listed, return False.
    best_offer = False if soup_pgn.find('a',{'id':'boBtn_btn'})==None else True
    
    # look at the banner in span with id=vi-ret-accrd-txt, if it says not accept, return False, else True.
    if soup_pgn.find('span',{'id':'vi-ret-accrd-txt'}):
        if soup_pgn.find('span',{'id':'vi-ret-accrd-txt'}).get_text().strip()=='Seller does not accept returns':
            returns_allowed = False
        else:
            returns_allowed = True
    else:
        returns_allowed = True 
    
    # look for the shipping summary in the span with id=shSummary, if it says Calculate, return True, else False
    shipping_cal = True if soup_pgn.find('span',{'id':'shSummary'}).find('a').get_text()=='Calculate' else False
    # if shipping needs to be calculated, shipping cost is None
    if shipping_cal:
        shipping_cost = None
    # if not, look at the span with id=fshippingCost, 
    # if it says FREE, shipping cost=0, if not, return the US cents value for shipping
    elif soup_pgn.find('span',{'id':'fshippingCost'}):
            if soup_pgn.find('span',{'id':'fshippingCost'}).get_text().strip()=='FREE':
                shipping_cost = 0
            else:
                raw_text = soup_pgn.find('span',{'id':'fshippingCost'}).get_text().strip().replace(',','')
                shipping_cost = int(float(re.search(r'[0-9]+.[0-9]+', raw_text)[0])*100)
    # if it says fast and free, shipping cost=0
    elif soup_pgn.find('span',class_='vi-fnf-ship-txt fnfgreen'):
        shipping_cost = 0
    # if it is not in USD, use id=convetedPriceId, and return shipping cost in US cents
    elif soup_pgn.find('span',{'id':'convetedPriceId'}):
        raw_text = soup_pgn.find('span',{'id':'convetedPriceId'}).get_text().strip().replace(',','')
        shipping_cost = int(float(re.search(r'[0-9]+.[0-9]+', raw_text)[0])*100)
    # other situations like Local pick ups and needs contact seller, return None.
    else:
        shipping_cost = None    
    
    
    info_keys = ['sponsored','item_id', 'item_title', 'item_price', 'condition', 'items_sold', 'seller_name', 'seller_score',
                 'best_offer', 'returns_allowed', 'shipping_cal', 'shipping_cost']
    info_vals = [sponsored, item_id, item_title, item_price, condition, items_sold, seller_name, seller_score, best_offer,
                 returns_allowed, shipping_cal, shipping_cost]
    item_dict = {info_keys[i]: info_vals[i] for i in range(len(info_keys))}
    
    return item_dict

In [3]:
# creat a function to get a list of files in the directory. 
# Due to the macOS settings, there is a .DS_store file that interrupts the compilation.
nonsponsored_files = []
for file in os.listdir(os.getcwd() + "/non-sponsored/"):
    if file.endswith('.htm'):
        nonsponsored_files.append(file)

In [298]:
# nonsponsored items' info using parallel computing
p = Pool(12)  # Processing 12 files at a time.
nonsponsored_items_info = p.map(extractInfo_non, nonsponsored_files)
p.terminate()
p.join()
nonsponsored_items_info

[{'sponsored': False,
  'item_id': 323574276084,
  'item_title': 'Canon EOS 5D Mark IV DSLR Camera with 24-105mm f/4L II Lens Bundle050',
  'item_price': 289074,
  'condition': 'New',
  'items_sold': None,
  'seller_name': '6ave',
  'seller_score': 63650,
  'best_offer': False,
  'returns_allowed': True,
  'shipping_cal': False,
  'shipping_cost': 0},
 {'sponsored': False,
  'item_id': 142380132444,
  'item_title': 'New Year Deal Sale 5Dm4 Canon Eos 5D Mark Iv Dslr Camera Body M4 - 1483C002',
  'item_price': 444800,
  'condition': 'New',
  'items_sold': None,
  'seller_name': 'barterboxusa',
  'seller_score': 1852,
  'best_offer': False,
  'returns_allowed': True,
  'shipping_cal': False,
  'shipping_cost': 0},
 {'sponsored': False,
  'item_id': 372814549865,
  'item_title': 'Canon EOS 5D Mark III Digital SLR Camera Body {22.3 M/P} (Count: 168,778)',
  'item_price': 97755,
  'condition': 'Used',
  'items_sold': None,
  'seller_name': 'kehoutlet',
  'seller_score': 99351,
  'best_offer'

## Get items into SQLDB

In [303]:
# Connect to the sql server
pw = "" # please insert your server password here
conn = pymysql.connect(host='localhost', user = 'root', password = pw)
cursor = conn.cursor()
# Create database eBay
SQL_DB = "eBay"
cursor.execute('DROP DATABASE IF EXISTS ' + SQL_DB)
query_DB = "CREATE DATABASE IF NOT EXISTS " + SQL_DB + ";"
cursor.execute(query_DB)
cursor.close()
conn.close()

In [305]:
# Create SQL DB and table, insert item information
conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS %s' % ('eBay_items'))

# Query to create table eBay_items in DB
sql = '''
        CREATE TABLE IF NOT EXISTS eBay_items
       (
       sponsored VARCHAR(255),
       item_id BIGINT PRIMARY KEY,
       item_title VARCHAR(255),
       item_price INT(11),
       conditions VARCHAR(255),
       items_sold INT(11),
       seller_name VARCHAR(255),
       seller_score INT(11),
       best_offer VARCHAR(255),
       returns_allowed VARCHAR(255),
       shipping_cal VARCHAR(255),
       shipping_cost INT(11)
       );
      '''
cursor.execute(sql)

# Query to insert data
q = """
    INSERT INTO eBay_items (sponsored, item_id, item_title, item_price, conditions, items_sold, seller_name, 
    seller_score, best_offer, returns_allowed, shipping_cal, shipping_cost)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
# loop through the list of dictionaries and insert data accordingly to the table
for item in sponsored_items_info:
    vals = (item['sponsored'],item['item_id'],item['item_title'],item['item_price'],item['condition'],
           item['items_sold'],item['seller_name'],item['seller_score'],item['best_offer'],item['returns_allowed'],
           item['shipping_cal'],item['shipping_cost'])
    cursor.execute(q, vals)
    conn.commit()

for item in nonsponsored_items_info:
    vals = (item['sponsored'],item['item_id'],item['item_title'],item['item_price'],item['condition'],
          item['items_sold'],item['seller_name'],item['seller_score'],item['best_offer'],item['returns_allowed'],
           item['shipping_cal'],item['shipping_cost'])
    cursor.execute(q, vals)
    conn.commit()

cursor.close()
conn.close()

## Summary stats on each item. 





In [359]:
def getSummaryStats():
    """
    The function runs queries to retrieve summary stats from mysql DB.
    
    RETURN: DataFrames of summary stats for AVG, STD, MIN and MAX.
    """
    
    conn = pymysql.connect(host='localhost', user = 'root', password = pw, database=SQL_DB)
    
    # getting AVG
    query_avg = """
                SELECT sponsored, conditions, 
                AVG(item_price) AS avg_price,
                AVG(items_sold) AS avg_sold,
                AVG(seller_score) AS avg_seller_score,
                AVG(best_offer) AS avg_best_offer,
                AVG(returns_allowed) AS avg_returns_allowed,
                AVG(shipping_cost) AS avg_shipping_cost
                FROM eBay_items
                GROUP BY sponsored, conditions
                ORDER BY sponsored, conditions
                """
    df_avg = pd.read_sql_query(query_avg, conn)
    
    # getting STD
    query_std = """
                SELECT sponsored, conditions, 
                STD(item_price) AS std_price,
                STD(items_sold) AS std_sold,
                STD(seller_score) AS std_seller_score,
                STD(best_offer) AS std_best_offer,
                STD(returns_allowed) AS std_returns_allowed,
                STD(shipping_cost) AS std_shipping_cost
                FROM eBay_items
                GROUP BY sponsored, conditions
                ORDER BY sponsored, conditions
                """
    df_std = pd.read_sql_query(query_std, conn)
    
    # getting MIN
    query_min = """
                SELECT sponsored, conditions, 
                MIN(item_price) AS min_price,
                MIN(items_sold) AS min_sold,
                MIN(seller_score) AS min_seller_score,
                MIN(best_offer) AS min_best_offer,
                MIN(returns_allowed) AS min_returns_allowed,
                MIN(shipping_cost) AS min_shipping_cost
                FROM eBay_items
                GROUP BY sponsored, conditions
                ORDER BY sponsored, conditions
                """
    df_min = pd.read_sql_query(query_min, conn)
    
    # getting MAX
    query_max = """
                SELECT sponsored, conditions, 
                MAX(item_price) AS max_price,
                MAX(items_sold) AS max_sold,
                MAX(seller_score) AS max_seller_score,
                MAX(best_offer) AS max_best_offer,
                MAX(returns_allowed) AS max_returns_allowed,
                MAX(shipping_cost) AS max_shipping_cost
                FROM eBay_items
                GROUP BY sponsored, conditions
                ORDER BY sponsored, conditions
                """
    df_max = pd.read_sql_query(query_max, conn)
    conn.close()
    
    return df_avg, df_std, df_min, df_max

In [360]:
df_AVG, df_STD, df_MIN, df_MAX = getSummaryStats()

In [361]:
# print AVG stats
df_AVG

,sponsored,conditions,avg_price,avg_sold,avg_seller_score,avg_best_offer,avg_returns_allowed,avg_shipping_cost
0,0,For parts or not working,41720.2000,NaN,4493.2000,0.466667,0.066667,1572.1818
1,0,Manufacturer refurbished,104950.0000,NaN,802.0000,0.500000,0.000000,NaN
2,0,New,306085.8926,2.8491,94937.9843,0.029083,0.941834,80.5869
3,0,Open box,243070.0000,NaN,16139.7059,0.352941,0.529412,707.0714
4,0,Seller refurbished,108402.2222,3.0000,44444.0000,0.222222,0.555556,251.4286
5,0,Used,107693.4768,1.4000,11007.6909,0.438384,0.551515,1267.2323
6,1,New,305000.1452,4.0000,270410.9032,0.048387,0.983871,32.7869
7,1,Open box,268102.3333,NaN,28927.3333,0.666667,0.666667,7500.0000
8,1,Seller refurbished,132838.3333,NaN,66237.3333,0.000000,0.666667,0.0000
9,1,Used,144557.5833,1.0000,22059.3542,0.395833,0.750000,1533.6977


In [362]:
# print STD stats
df_STD

,sponsored,conditions,std_price,std_sold,std_seller_score,std_best_offer,std_returns_allowed,std_shipping_cost
0,0,For parts or not working,31159.199791,NaN,11218.449211,0.498888,0.249444,1534.942037
1,0,Manufacturer refurbished,5050.000000,NaN,432.000000,0.500000,0.000000,NaN
2,0,New,93608.818116,2.157884,126930.032112,0.168039,0.234056,619.351309
3,0,Open box,102057.950809,NaN,26506.206287,0.477885,0.499134,1245.096122
4,0,Seller refurbished,57070.595180,0.000000,49113.004044,0.415740,0.496904,615.871707
5,0,Used,77410.851078,0.800000,34306.373050,0.496189,0.497339,2292.406696
6,1,New,79079.468925,3.043544,228028.502792,0.214583,0.125972,253.966121
7,1,Open box,45109.405703,NaN,26255.480054,0.471405,0.471405,7500.000000
8,1,Seller refurbished,119018.359531,NaN,46829.796500,0.000000,0.471405,0.000000
9,1,Used,105211.475736,0.000000,52166.315114,0.489029,0.433013,2284.609137


In [363]:
# print MIN stats
df_MIN

,sponsored,conditions,min_price,min_sold,min_seller_score,min_best_offer,min_returns_allowed,min_shipping_cost
0,0,For parts or not working,9999,NaN,0,0,0,0.0
1,0,Manufacturer refurbished,99900,NaN,370,0,0,NaN
2,0,New,1587,1.0,1,0,0,0.0
3,0,Open box,2820,NaN,9,0,0,0.0
4,0,Seller refurbished,31160,3.0,9,0,0,0.0
5,0,Used,14940,1.0,0,0,0,0.0
6,1,New,48000,1.0,0,0,0,0.0
7,1,Open box,204308,NaN,174,0,0,0.0
8,1,Seller refurbished,32205,NaN,10,0,0,0.0
9,1,Used,22900,1.0,2,0,0,0.0


In [364]:
# print MAX stats
df_MAX

,sponsored,conditions,max_price,max_sold,max_seller_score,max_best_offer,max_returns_allowed,max_shipping_cost
0,0,For parts or not working,127999,NaN,46062,1,1,3600.0
1,0,Manufacturer refurbished,110000,NaN,1234,1,0,NaN
2,0,New,640000,10.0,916794,1,1,7500.0
3,0,Open box,468900,NaN,63650,1,1,3500.0
4,0,Seller refurbished,198599,3.0,99351,1,1,1760.0
5,0,Used,800000,3.0,166201,1,1,15000.0
6,1,New,540733,10.0,568352,1,1,2000.0
7,1,Open box,300000,NaN,63650,1,1,15000.0
8,1,Seller refurbished,300000,NaN,99351,0,1,0.0
9,1,Used,384600,1.0,165739,1,1,7500.0


By looking at the summary stats, on average, sponsored items tend to have higher seller score, have higher shipping cost, and to be priced higher and more generous offering free returns. <br>

The minimum price of sponored items is always higher than that of the non-sponsored items, while the max price being noticeably lower. <br>

To predict the sponsor/non-sponsor items (in this case the Canon EOS 5D DSLR Camera), the min/max item price, seller score, and whether offering returns are some good predictor features after initial glance at the data statistics.

